In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.3.1'

In [3]:
# 创建变量
v = tf.Variable(0.0)

In [4]:
(v + 1).numpy()

1.0

In [5]:
# 改变变量值
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [6]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [7]:
# 加一
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [8]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [9]:
# 读取变量值
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [10]:
# 自动微分运算(只能计算float类型的数据)
w = tf.Variable([[1.0]])
with tf.GradientTape() as t: # 上下文管理器
    loss = w * w

In [11]:
# 求解loss对w的微分/梯度
grad = t.gradient(loss,w)
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [12]:
# 自动微分运算
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w) # t跟踪w的运算
    loss = w * w

In [13]:
dloss_dw = t.gradient(loss,w)

In [14]:
dloss_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [15]:
# 自动微分运算
w = tf.constant(3.0)
with tf.GradientTape(persistent = True) as t: # persistent = True将微分持久记录，可以多次计算微分
    t.watch(w) # t跟踪w的运算
    y = w * w
    z = y * y

In [16]:
dy_dx = t.gradient(y,w)
dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [17]:
dz_dw = t.gradient(z, w)
dz_dw

<tf.Tensor: shape=(), dtype=float32, numpy=108.0>

自定义训练

In [42]:
(train_images,train_labels) ,(test_images,test_labels) = tf.keras.datasets.mnist.load_data()

In [43]:
train_images.shape

(60000, 28, 28)

In [45]:
# 改变数据类型
train_images = tf.cast(train_images/255,tf.float32)
test_images = tf.cast(test_images/255,tf.float32)

In [46]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [48]:
train_images = tf.expand_dims(train_images,-1) # 扩张维度，扩张最后一列为1
test_images = tf.expand_dims(test_images,-1) # 扩张维度，扩张最后一列为1

In [49]:
train_labels

<tf.Tensor: shape=(60000,), dtype=int64, numpy=array([5, 0, 4, ..., 5, 6, 8], dtype=int64)>

In [50]:
dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images,test_labels))

In [51]:
dataset,test_dataset

(<TensorSliceDataset shapes: ((28, 28, 1, 1), ()), types: (tf.float32, tf.int64)>,
 <TensorSliceDataset shapes: ((28, 28, 1, 1), ()), types: (tf.float32, tf.int64)>)

In [52]:
dataset = dataset.shuffle(10000).batch(32) # 不设置repeat()时默认会重复一次
test_dataset = test_dataset.batch(32)

In [53]:
dataset,test_dataset

(<BatchDataset shapes: ((None, 28, 28, 1, 1), (None,)), types: (tf.float32, tf.int64)>,
 <BatchDataset shapes: ((None, 28, 28, 1, 1), (None,)), types: (tf.float32, tf.int64)>)

In [54]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(None,None,1)), # (None,None,1)表示图片大小任意
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [55]:
# 自定义循环
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # 未激活，from_logits=True
features, labels = next (iter(dataset)) # 取出第一个批次的数据

In [56]:
predictions  = model(features)

In [57]:
predictions.shape

TensorShape([32, 10])

In [58]:
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int64)>

In [59]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([2, 7, 3, 4, 5, 4, 0, 7, 4, 0, 9, 7, 2, 6, 9, 4, 9, 5, 2, 8, 1, 1,
       4, 5, 3, 9, 0, 2, 9, 4, 9, 5], dtype=int64)>

In [60]:
# 初始化计算对象
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy') 
test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy') 

In [61]:
# 定义损失函数，loss返回交叉熵损失
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y,y_) # y_为预测值

In [62]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels,pred)
        #loss_step = loss(model,images,labels) # 每一步的损失值
    grads = t.gradient(loss_step,model.trainable_variables) # 损失函数与可训练参数之间的梯度
    optimizer.apply_gradients(zip(grads,model.trainable_variables)) # 优化函数应用梯度进行优化
    # 汇总计算平均loss
    train_loss(loss_step)
    # 汇总计算平均acc
    train_accuracy(labels,pred)

In [63]:
def test_step(model,images,labels):
        pred = model(images)
        loss_step = loss_func(labels,pred)
        # 汇总计算平均loss
        test_loss(loss_step)
        # 汇总计算平均acc
        test_accuracy(labels,pred)

In [64]:
def train():
    for epoch in range(10):
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print('Epoch{}  loss is {}, accuracy is {}'.format(epoch,train_loss.result(),train_accuracy.result()))
        for (batch,(images,labels)) in enumerate(test_dataset):
            test_step(model,images,labels)
        print('Epoch{}  test_loss is {}, test_accuracy is {}'.format(epoch,test_loss.result(),test_accuracy.result()))
        # 重置
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [65]:
train()

Epoch0  loss is 2.301481008529663, accuracy is 0.11178333312273026
Epoch0  test_loss is 2.3004038333892822, test_accuracy is 0.11349999904632568
Epoch1  loss is 2.256425619125366, accuracy is 0.17403332889080048
Epoch1  test_loss is 2.1326065063476562, test_accuracy is 0.22669999301433563
Epoch2  loss is 2.0264012813568115, accuracy is 0.2898666560649872
Epoch2  test_loss is 1.9405102729797363, test_accuracy is 0.34360000491142273
Epoch3  loss is 1.896636724472046, accuracy is 0.368966668844223
Epoch3  test_loss is 1.824200987815857, test_accuracy is 0.4156999886035919
Epoch4  loss is 1.7756662368774414, accuracy is 0.42524999380111694
Epoch4  test_loss is 1.6926041841506958, test_accuracy is 0.43950000405311584
Epoch5  loss is 1.635901927947998, accuracy is 0.4648166596889496
Epoch5  test_loss is 1.560739278793335, test_accuracy is 0.4733000099658966
Epoch6  loss is 1.501874327659607, accuracy is 0.49818333983421326
Epoch6  test_loss is 1.4185281991958618, test_accuracy is 0.516399979

tf.kears.metrics汇总计算模块

In [ ]:
# 计算均值
m = tf.keras.metrics.Mean('acc')

In [ ]:
# 得到计算均值的结果
m.result()

In [ ]:
# 重置均值计算状态：0
m.reset_states()

In [ ]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [ ]:
# 自动选出model(features)中的最大值与labels进行比较
a(labels,model(features))